# Ergonomic pose fine-tuning: 

## fine tuning yolo for our classification task

- option 1: freeze the complete model, use it as a feature extractor and add a classification head to be trained on our dataset
- option 2: cut the keypoint output layer, freeze the rest of the model and add a classification head to be trained on our dataset

### 1) YOLOv8pose model as feature extractor

imports:

In [ ]:
# early stopping bei hoher num_epochs (300)
# batch_size festlegen (8, 16?)

# HPO:
# momentum: [0.9, 0.95]
# lr: [1e-4, 5e-5]

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
#%pip install -U ultralytics
import torch
from functools import partial

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")
    
get_device()

### load the model

strangely the yolo model does not like to be frozen and put to eval mode (then it loads some own dataset):

In [ ]:
# we had to outcomment this + yolomodel.eval() later
#for param in model.parameters():
#    param.requires_grad = False # freeze the YOLO model

#for name, param in model.named_parameters():
    #print(f"{name} is {'not ' if param.requires_grad else ''}frozen"

defining a classification layer:

In [ ]:
import torch
import torch.nn as nn

# `feature_extractor` is the part of the model suitable for feature extraction

class YOLO_kp_Classifier(nn.Module):
    def __init__(self, num_keypoints, num_classes=3):
        super(YOLO_kp_Classifier, self).__init__()
        
        # add classification layer to the model
        self.classifier = nn.Sequential(
            nn.Linear(num_keypoints*2, 512),  # 17 keypoints * 2 (x, y coordinates for each keypoint)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, keypoints_flattened):
        
        output = self.classifier(keypoints_flattened)
        return output


initializing the model:

In [ ]:
# initialize the classifier model
num_classes = 7  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model

kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)

load the own dataset:

In [ ]:
from torch.utils.data import random_split, DataLoader, Subset, TensorDataset
from torchvision import datasets, transforms

In [ ]:
# Normalization should be applied to the keypoints not the images I think - Vipin

# Custom transform because YOLO likes Numpy Arrays / CV2 Images,
# it automatically resizes it to its preffered size, and converts
# the output back to the source size, better for precision and normalization
from sympy import im


class PILToNumpyTransform:
    def __call__(self, pil_img):
        """
        Convert a PIL Image to an OpenCV Image / Numpy Array.

        Parameters:
            pil_img (PIL.Image): The PIL Image to be converted.

        Returns:
            np.ndarray: The converted OpenCV Image in RGB format.
        """
        # Convert PIL Image to NumPy array (RGB)
        img_array = np.array(pil_img)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

        return img_array
    
data_transforms = transforms.Compose([
    transforms.Resize((640, 480)), # Resize images to 640x640
    transforms.ToTensor() # Convert to tensor
])

In [ ]:
data_path = 'E:/Users/Vipin\Documents/BHT/3. Semester/Learning from images/Pose Dataset Complete'
own_dataset = datasets.ImageFolder(root=data_path, transform=data_transforms)

In [ ]:
own_dataset

splitting the data into train, validation and test set:

In [ ]:
generator1 = torch.Generator().manual_seed(13)  # set seed for reproducibility of the split
train_and_val_dataset, test_dataset = random_split(own_dataset, [0.8, 0.2], generator=generator1)  # 80% training and evaluation, 20% testing

feature extraction with yolo:

In [ ]:
def get_keypoints_from_yolo(model, inputs):
    with torch.no_grad(): # gradients are not computed for the frozen model
        results = []
        for input in inputs:
            result = model(input.unsqueeze(0))[0].keypoints.xy[0].to(input.device)
            print(result.shape)
            if result.shape[0] < 17:
                result = torch.cat([result, torch.zeros(17 - result.shape[0], 2).to(input.device)], dim = 0)
            
            results.append(result)
    return torch.stack(results)

processing keypoints:

In [ ]:
# todo: check if this is sufficient, otherwise try z-score normalization or other methods

# todo: !!!!!!!!! check correct position to use it / get min max earlier !!!!!!!!! needs to be the same normalization for all the data

def process_keypoints_for_classifier(keypoints):
    # Calculate mean and standard deviation with keepdim=True to preserve broadcasting compatibility
    mean_vals = keypoints.mean(dim=1, keepdim=True)
    std_vals = keypoints.std(dim=1, keepdim=True) + 1e-6  # Adding a small epsilon to prevent division by zero

    # Perform standardization
    kp_tensor_standardized = (keypoints - mean_vals) / std_vals
    
    # Flatten the last two dimensions while keeping the batch dimension
    batch_flattened = kp_tensor_standardized.view(keypoints.size(0), -1)
    
    return batch_flattened

#### training the model:

- with cross-entropy loss function (fits our classification task)
- Adam optimizer

HPO:

In [ ]:
#!pip install --upgrade pip
#%pip install scikit-learn
#%pip install optuna
import optuna
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

In [ ]:
# initialize the classifier model
num_classes = 7  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model
# loss function and optimizer
criterion = nn.CrossEntropyLoss()

In [ ]:
def preprocess_dataset(yolo_model, dataset, batch_size, device):
    processed_keypoints_list = []
    labels_list = []
    
    # Assuming `dataset` is an iterable of (input, label) pairs
    for inputs, labels in dataset:
        inputs = inputs.to(device)
        keypoints = get_keypoints_from_yolo(yolo_model, inputs)  # Extract keypoints        
        processed_kps = process_keypoints_for_classifier(keypoints)  # Process keypoints        
        processed_keypoints_list.append(processed_kps)
        labels_list.append(labels)

    # Convert lists to tensors
    processed_keypoints_tensor = torch.cat(processed_keypoints_list, dim=0)
    labels_tensor = torch.cat(labels_list, dim=0)
    
    # Create a new TensorDataset and DataLoader
    preprocessed_dataset = TensorDataset(processed_keypoints_tensor, labels_tensor)
    preprocessed_loader = DataLoader(preprocessed_dataset, batch_size=batch_size, shuffle=True)
    
    return preprocessed_loader
    
def train_and_eval_model(kp_classifier_model, yolo_model, optimizer, num_epochs, train_loader, val_loader, device):
    
    kp_classifier_model = kp_classifier_model.to(device)  # move model to device
    yolo_model = yolo_model.to(device)  # move YOLO model to device
    kp_classifier_model.train()  # set model to training mode
    
    print("type of train_loader", type(train_loader))

    for epoch in range(num_epochs):
        for batch in train_loader:
            # access images and labels
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            #print("inputs", type(inputs), len(inputs), inputs)
            #inputs, labels = inputs.to(device), labels.to(device)  # move data to device
            
            # keypoints = get_keypoints_from_yolo(yolo_model, inputs) # get keypoints from the YOLO model
            # processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
            #print("keypoints", type(keypoints), len(keypoints[0]), keypoints)
            #print("processed_kps", type(processed_kps), len(processed_kps[0]), processed_kps)
            
            optimizer.zero_grad()  # Zero the parameter gradients
            classification_output = kp_classifier_model(inputs) # get results for the classification
            loss = criterion(classification_output, labels)  # Compute loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update weights

            # todo: further processing, such as calculating accuracy or loss, goes here

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            
        
        kp_classifier_model.eval()  # model to evaluation mode

        total_loss = 0
        all_predictions = []
        all_labels = []

        with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
            for inputs, labels in val_loader:  # iterate over validation dataset
                inputs, labels = inputs.to(device), labels.to(device)  # move data to device
                
                # Not necessary because already preprocessed
                # keypoints = get_keypoints_from_yolo(yolo_model, inputs) # get keypoints from the YOLO model
                # processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
                
                classification_output = kp_classifier_model(inputs) # get results for the classification 
                loss = criterion(classification_output , labels)  # compute loss
                total_loss += loss.item()  # accumulate the loss
                # get predictions for output
                _, predicted = torch.max(classification_output.data, 1)
                # collect the predictions and labels
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            # calculate the validation metrics:
            avg_loss = total_loss / len(val_loader)  # get the average loss
            accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
            f1 = f1_score(all_labels, all_predictions, average='weighted')  
            conf_matrix = confusion_matrix(all_labels, all_predictions)
            
            print(f"Validation Loss: {avg_loss:.4f}")
            print(f"Validation Accuracy: {accuracy:.4f}")
            print(f"Validation F1 Score: {f1:.4f}")
            print("Confusion Matrix:")
            print(conf_matrix)
            
            # here: F1 score chosen as the metric to optimize
            # other options: - combining metrics like accuracy and F1 score to maximize on both
            #                        - or multi-objective optimization on F1 score and accuracy
    return f1 

In [ ]:
# this function was generated by gpt-4

def get_k_fold_indices(n, k=5, random_seed=None):
    """
    Generate indices for k-fold cross-validation.

    Parameters:
    - n: Total number of samples in the dataset.
    - k: Number of folds.
    - random_seed: Optional seed for reproducibility.

    Returns:
    - A list of tuples, each containing (train_indices, val_indices) for a fold.
    """
    # Initialize the random generator
    g = torch.Generator()
    if random_seed is not None:
        g.manual_seed(random_seed)
    
    # Generate a random permutation of indices
    indices = torch.randperm(n, generator=g).tolist()
    
    # Calculate fold sizes
    fold_sizes = [n // k for _ in range(k)]
    for i in range(n % k):
        fold_sizes[i] += 1
    
    # Generate train and validation indices for each fold
    current = 0
    folds_indices = []
    for fold_size in fold_sizes:
        start, end = current, current + fold_size
        val_indices = indices[start:end]
        train_indices = indices[:start] + indices[end:]
        folds_indices.append((train_indices, val_indices))
        current = end
    
    return folds_indices


In [ ]:
def objective(trial, dataset, yolo_model):
    # Define hyperparameters to optimize
    lr = trial.suggest_categorical("lr", [1e-4, 5e-5]) #, 5e-4, 1e-3])
    #momentum = trial.suggest_categorical("momentum", [0.9, 0.95])
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_epochs = trial.suggest_categorical("num_epochs", [100])

    validation_scores = []

    n = len(dataset)
    k = 5
    folds_indices = get_k_fold_indices(n, k, random_seed=13)
    device = get_device()
    
    for fold, (train_idx, val_idx) in enumerate(folds_indices, start=1):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        print("type of train subset: " , type(train_subset))
        train_loader = DataLoader(train_subset, batch_size=1, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=1, shuffle=False)
        
        train_loader = preprocess_dataset(yolo_model, train_loader, batch_size, device)
        val_loader = preprocess_dataset(yolo_model, val_loader, batch_size, device)
        print("type of train loader: " , type(train_loader))
        # initializing the model and optimizer with the chosen hyperparameters
        kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)
        optimizer = torch.optim.Adam(kp_classifier_model.parameters(), lr=lr, weight_decay=1e-5)
    
        # training and evaluating the model
        validation_score = train_and_eval_model(kp_classifier_model, yolo_model, optimizer, num_epochs, train_loader, val_loader, device)
        validation_scores.append(validation_score)

    return np.mean(validation_scores)


In [ ]:
yolo_model = YOLO('yolov8n-pose.pt')
optimize = partial(objective, dataset = train_and_val_dataset, yolo_model = yolo_model)

study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=20)  # run 20 trials

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

In [ ]:
best_params = study.best_params
best_params

In [ ]:
best_params = study.best_params
# extract best_parameters
batch_size = best_params['batch_size']
lr = best_params['lr']
# momentum = best_params['momentum']
num_epochs = best_params['num_epochs']

#### train and evaluate final model on test set:

Unfortunately Optuna cannot output the best model, so we train again on the combined train and validation set with the best parameters found.

dataloader for final evaluation:

In [ ]:
train_and_eval_loader = DataLoader(train_and_val_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

train_and_eval_loader = preprocess_dataset(yolo_model, train_and_eval_loader, batch_size, get_device())
test_loader = preprocess_dataset(yolo_model, test_loader, batch_size, get_device())

final evaluation on test set:

In [ ]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(kp_classifier_model.classifier.parameters(), lr=lr, weight_decay=1e-5)

# new instance of the model:
kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)
kp_classifier_model = kp_classifier_model.to(get_device())  # move model to device
kp_classifier_model.train()  # set model to training mode

for epoch in range(num_epochs):
    for inputs, labels in train_and_eval_loader:  # Assuming data_loader is your DataLoader instance for the dataset
        inputs, labels = inputs.to(get_device()), labels.to(get_device())  # move data to device
        optimizer.zero_grad()  # Zero the parameter gradients
        classification_output = kp_classifier_model(inputs) # get results for the classification 
        loss = criterion(classification_output , labels)  # Compute loss
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update weights

        # todo: further processing, such as calculating accuracy or loss, goes here

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
    
    kp_classifier_model.eval()  # model to evaluation mode

    total_loss = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
        for inputs, labels in test_loader:  # iterate over validation dataset
            inputs, labels = inputs.to(get_device()), labels.to(get_device())  # move data to device
            classification_output = kp_classifier_model(inputs) # get results for the classification 
            loss = criterion(classification_output , labels)  # compute loss
            total_loss += loss.item()  # accumulate the loss
            _, predicted = torch.max(classification_output.data, 1)
            # collect the predictions and labels
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        # calculate the validation metrics:
        avg_loss = total_loss / len(test_loader)  # get the average loss
        accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
        f1 = f1_score(all_labels, all_predictions, average='weighted')  
        conf_matrix = confusion_matrix(all_labels, all_predictions)
        
        print(f"Test Loss: {avg_loss:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")
        print(f"Test F1 Score: {f1:.4f}")
        print("Confusion Matrix:")
        print(conf_matrix)